<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Dylan/Validation_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Install CaImAn - takes around 2 minutes

!git clone https://github.com/flatironinstitute/CaImAn.git
%cd '/content/CaImAn/'
!pip install -e .

# Install caiman dependencies (&> /dev/null will suppress the hundreds of printed lines in the output)
!pip install -r requirements.txt &> /dev/null

#import other dependencies
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio

#IMPORTANT! Newer versions of h5py will cause errors when saving results
!pip install h5py==2.10.0
import h5py

#Set up caiman
!python setup.py build_ext -i

#Other file setup
!python caimanmanager.py install --inplace

#Caiman imports
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.summary_images import local_correlations_movie_offline
from scipy.ndimage import center_of_mass
from IPython.display import display, clear_output

Cloning into 'CaImAn'...
remote: Enumerating objects: 24916, done.
remote: Counting objects: 100% (851/851), done.
remote: Compressing objects: 100% (421/421), done.
remote: Total 24916 (delta 475), reused 739 (delta 414), pack-reused 24065
Receiving objects: 100% (24916/24916), 518.52 MiB | 29.09 MiB/s, done.
Resolving deltas: 100% (16688/16688), done.
Checking out files: 100% (317/317), done.
/content/CaImAn
Obtaining file:///content/CaImAn
  Running setup.py develop for caiman
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
running build_ext
Installed /root/caiman_data


## Get paths to movie files and labelled ROI masks

In [2]:
#get a list of our masks and a list of our stacks in the same order
from os import listdir
maskpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs'
mask_filenames = [f for f in listdir(maskpath) if 'CLEAN' not in f and '.ipynb' not in f]
mask_filenames

['210815_0_1_manualROIs_mix1_syt.mat',
 '211106_1_1_manualROIs_tm4_syt.mat',
 '211023_0_1_manualROIs_tm2_tm4_syt.mat',
 '210731_0_1_manualROIs_t5_syt.mat',
 '210728_0_1_manualROIs_tm2_tm9_syt.mat',
 '211106_0_1_manualROIs_tm1_syt.mat']

In [17]:
stack_indices = [x[:10] for x in mask_filenames] #get the index portion of the masks
stack_celltypes = [x[22:(len(x)-4)] for x in mask_filenames] #get the cell type descriptions
stack_filenames = ['CLEAN_' + stack_indices[x] + '_stackRaw_mc_' + stack_celltypes[x] \
                   + '_.h5' for x in range(len(mask_filenames))] #reconstruct filenames for the movies we have masks for
stack_filenames

['CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5',
 'CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5',
 'CLEAN_211023_0_1_stackRaw_mc_tm2_tm4_syt_.h5',
 'CLEAN_210731_0_1_stackRaw_mc_t5_syt_.h5',
 'CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5',
 'CLEAN_211106_0_1_stackRaw_mc_tm1_syt_.h5']

In [18]:
#checking that files exist in our cleaned file - only 2 do
stackpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED'
stackfiles = [f for f in listdir(stackpath) if f in stack_filenames]
stackfiles

['CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5',
 'CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5',
 'CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']

In [22]:
#convert filenames back into paths
maskpaths = [maskpath+'/'+f for f in mask_filenames]
stackpaths = [stackpath+'/'+f for f in stackfiles]

In [23]:
#for now only keep the masks we cleaned have movies for - also in the correct order to match up
maskpaths = [maskpaths[x] for x in [1, 0, 4]]
maskpaths

['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211106_1_1_manualROIs_tm4_syt.mat',
 '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat',
 '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat']

In [24]:
#join the lists into pairs of tuples
mask_stack_pairs = list(map(lambda x, y:[x,y], maskpaths, stackpaths))
mask_stack_pairs

[['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211106_1_1_manualROIs_tm4_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']]

## Initialize parameters object with starter values

In [8]:
#create parameters object
opts = params.CNMFParams()
#fname will be assigned in the loop
fnames = []
subfolder = 'stackRaw_mc'

In [9]:
# set up some parameters for extraction
#fnames = path to video file, set above
                        # file(s) to be analyzed
is_patches = True       # flag for processing in patches or not - CONFIRMED FROM CARL
fr = 20                 # approximate frame rate of data - CONFIRMED FPS
decay_time = .4        # length of transient - CONFIRMED APPROPRIATE FOR OUR INDICATOR GCaMP6f
dims = [128, 256]

if is_patches:          # PROCESS IN PATCHES AND THEN COMBINE - not used
    rf = 25             # half size of each patch - not used
    stride = 5          # overlap between patches - not used
    K = 3               # number of components in each patch - not used
else:                   # PROCESS THE WHOLE FOV AT ONCE
    rf = None           # setting these parameters to None - CONFIRMED
    stride = None       # will run CNMF on the whole FOV - CONFIRMED
    K = 20              # number of neurons expected (in the whole FOV) - TUNE

gSig = [4, 4]           # expected half size of neurons - TUNE
merge_thresh = 0.9     # merging threshold, max correlation allowed - TUNE
p = 1                   # order of the autoregressive system - From carl's code, probably should be 1
gnb = 2                 # global background order - TUNE

opts.set('data', {'fnames': fnames,
                   'fr': fr,
                   'decay_time': decay_time,
                   'dims': dims,
                   'rf': rf,
                   'stride': stride,
                   'K': K,
                   'gSig': gSig,
                   'merge_thr': merge_thresh,
                   'p': p,
                   'nb': gnb
                  })


# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier (this will pick up only neurons
#           and filter out active processes)


#Not sure if these should be tuned or not

min_SNR = 2.5      # peak SNR for accepted components (if above this, acept)
rval_thr = 0.9     # space correlation threshold (if above this, accept)
use_cnn = False      # use the CNN classifier
min_cnn_thr = 0.9  # if cnn classifier predicts below this value, reject
cnn_lowest = 0.1 # neurons with cnn probability lower than this value are rejected

opts.set('quality', {'min_SNR': min_SNR,
                                'rval_thr': rval_thr,
                                'use_cnn': use_cnn,
                                'min_cnn_thr': min_cnn_thr,
                                'cnn_lowest': cnn_lowest})

#Manually assign subfolder variable
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder

#for development purposes heavily downsample to save time
opts.init['ssub'] = 5
opts.init['tsub'] = 5

## Define pipeline function to run Caiman

In [10]:
#Function to run cnmf, run seeded cnmf using masks, then return validation

def score_params(path_to_stack, path_to_masks, opts):
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)

  fnames = [path_to_stack]

  try:
    
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)
    
    #Run CNMF on raw stack using passed params
    cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
    print('cnm object initialized')
    cnm = cnm.fit_file(motion_correct = False, include_eval=True)
    print('cnmf and component evaluation completed')


    #Read in masks and reformat
    g = h5py.File(path_to_masks, 'r')

    #transpose the matrix and save to an array A
    mask_A = g['bwMaskStack'][:].T

    g.close()

    #rearrange the dimensions and show the new shape
    mask_A = mask_A.transpose(1,0,2)

    #reshape to 2D, first dimension is 128*256 (32768), 2nd dimension is the # of ROI's
    mask_A = mask_A.reshape((mask_A.shape[1]*mask_A.shape[0]), mask_A.shape[2])

    #convert the values from 0/1 to boolean False/True
    mask_A = np.array(mask_A, dtype=bool)
    print('mask read in and reformatted')

    dview.terminate()

    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)

    #Seeded CNMF only works when seeded using mmap
    import pathlib

    #create memmory map location for the original movie
    fname_new = cm.save_memmap(fnames, base_name=pathlib.Path(fnames[0]).stem + "_memmap_", order='C')

    #read data from mmap location
    Yr, dims, T = cm.load_memmap(fname_new)

    # load frames in python format (T x X x Y)
    mov = np.reshape(Yr.T, [T] + list(dims), order='F')
    print('movie matrix loaded for seeded cnmf')


    #For seeded CNMF, need to adjust some params
    rf = None
    only_init = False

    opts.patch['only_init'] = only_init
    opts.patch['rf'] = rf

    print('params adjusted for seeded cnmf')
    
    #Initialize a new cnmf object and pass in our masks as the "Ain" param
    #"Ain" is A-in, meaning the A matrix holding the spatial footprints of the roi's
    cnm_seeded = cnmf.CNMF(n_processes, params=opts, dview=dview, Ain=mask_A)
    print('seeded cnmf object initialized')
    cnm_seeded.fit(mov)
    print('seeded cnmf completed')

    #Try running register_ROIs and see what happens
    matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance, A2 = cm.base.rois.register_ROIs(cnm_seeded.estimates.A, cnm.estimates.A, dims=dims)
    print('validation and scoring completed')

    #terminate cluster
    dview.terminate()

    #return performance metrics
    return {'matched_ROIs1': matched_ROIs1,
            'matched_ROIs2': matched_ROIs2,
            'non_matched1': non_matched1,
            'non_matched2': non_matched2, 
            'performance': performance,
            'A2': A2}

  except:
    print('failed')
    dview.terminate()

## Eventually, adjust params within loops to implement gridsearch

### For now, just loop through the 3 pairs at the moment and print accuracy results

In [11]:
#loop through the stack-mask pairs and print the accuracy results


#this will all be wrapped in another function that sets parameters
#this loop will run all stacks through caiman using a given set of parameters to get the accuracy
for pair in mask_stack_pairs:
  path_to_stack = pair[1]
  path_to_masks = pair[0]

  print(path_to_stack)
  print(path_to_masks)


  opts.data['fnames'] = [path_to_stack]
  print(opts.data['fnames'])

  results = score_params(path_to_stack=path_to_stack, \
             path_to_masks=path_to_masks, \
             opts=opts)
  
  #need to add storage to save results
  #could adjust this to only output the average performance at the end of a loop
  print(results['performance'])


../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5
../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5']
cnm object initialized


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


cnmf and component evaluation completed
mask read in and reformatted
movie matrix loaded for seeded cnmf
params adjusted for seeded cnmf
seeded cnmf object initialized
spatial support for each components given by the user
seeded cnmf completed
validation and scoring completed
{'recall': 0.0, 'precision': 0.0, 'accuracy': 0.0, 'f1_score': 0.0}
../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5
../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211106_1_1_manualROIs_tm4_syt.mat
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5']
cnm object initialized


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/content/CaImAn/caiman/source_extraction/cnmf/spatial.py", line 413, in regression_ipyparallel
    a_lrs = clf.fit(np.array(c.T), np.ravel(y))


failed


TypeError: ignored

## Old working code

In [17]:
# # cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
# cnm = cnm.fit_file(motion_correct = False, include_eval=True)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


In [18]:
# #Read in masks and reformat
# path_to_masks = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat'
# g = h5py.File(path_to_masks, 'r')

# #transpose the matrix and save to an array A
# mask_A = g['bwMaskStack'][:].T

# g.close()

# #rearrange the dimensions and show the new shape
# mask_A = mask_A.transpose(1,0,2)

# #reshape to 2D, first dimension is 128*256 (32768), 2nd dimension is the # of ROI's
# mask_A = mask_A.reshape((mask_A.shape[1]*mask_A.shape[0]), mask_A.shape[2])

# #convert the values from 0/1 to boolean False/True
# mask_A = np.array(mask_A, dtype=bool)
# print('mask read in and reformatted')

mask read in and reformatted


In [19]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# #Seeded CNMF only works when seeded using mmap
# import pathlib

# #create memmory map location for the original movie
# fname_new = cm.save_memmap(fnames, base_name=pathlib.Path(fnames[0]).stem + "_memmap_", order='C')
#  #read data from mmap location
# Yr, dims, T = cm.load_memmap(fname_new)

# # load frames in python format (T x X x Y)
# mov = np.reshape(Yr.T, [T] + list(dims), order='F')
# print('movie matrix loaded for seeded cnmf')

# dview.terminate()

# c, dview, n_processes = cm.cluster.setup_cluster(
#     backend='local', n_processes=None, single_thread=False)

# #For seeded CNMF, need to adjust some params
# rf = None
# only_init = False

# opts.patch['only_init'] = only_init
# opts.patch['rf'] = rf

# print('params adjusted for seeded cnmf')
  
# #Initialize a new cnmf object and pass in our masks as the "Ain" param
# #"Ain" is A-in, meaning the A matrix holding the spatial footprints of the roi's
# cnm_seeded = cnmf.CNMF(n_processes, params=opts, dview=dview, Ain=mask_A)
# print('seeded cnmf object initialized')
# cnm_seeded.fit(mov)
# print('seeded cnmf completed')

movie matrix loaded for seeded cnmf
params adjusted for seeded cnmf
seeded cnmf object initialized
spatial support for each components given by the user
seeded cnmf completed


/usr/local/lib/python3.7/dist-packages/scipy/sparse/dia.py:300: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


In [21]:
# matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance, A2 = cm.base.rois.register_ROIs(cnm_seeded.estimates.A, cnm.estimates.A, dims=dims)
# performance

{'accuracy': 0.05128205128205128,
 'f1_score': 0.0975609756097561,
 'precision': 0.06896551724137931,
 'recall': 0.16666666666666666}

In [25]:
# dview.terminate()

In [30]:
# results = score_params(path_to_stack=fnames[0], \
#              path_to_masks='../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat', \
#              opts=opts)
# results['performance']

cnm object initialized


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


cnmf and component evaluation completed
mask read in and reformatted
movie matrix loaded for seeded cnmf
params adjusted for seeded cnmf
seeded cnmf object initialized
spatial support for each components given by the user


/usr/local/lib/python3.7/dist-packages/scipy/sparse/dia.py:300: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


seeded cnmf completed
validation and scoring completed


{'accuracy': 0.05128205128205128,
 'f1_score': 0.0975609756097561,
 'precision': 0.06896551724137931,
 'recall': 0.16666666666666666}

In [ ]:
# dview.terminate()